"""
# 🤖 SIMAC - Sistema Inteligente de Monitoramento Contábil 🤖

**Apresentação:**

O SIMAC é um sistema inovador de monitoramento contábil desenvolvido para auxiliar profissionais da área na tarefa crucial de acompanhar as constantes mudanças na legislação tributária e obrigações acessórias. Em um ambiente regulatório dinâmico como o brasileiro, manter-se atualizado é um desafio complexo e que demanda tempo e atenção. O SIMAC surge como uma solução inteligente para otimizar esse processo, proporcionando maior eficiência e reduzindo o risco de erros e não conformidades.

**Importância:**

A importância do SIMAC reside em sua capacidade de:

* **Automatizar a Busca por Atualizações:** O sistema utiliza agentes inteligentes para vasculhar fontes oficiais e confiáveis (como Receita Federal, SEFAZ, Diário Oficial, etc.) em busca de novas normas e alterações. Isso elimina a necessidade de o profissional realizar essa busca manualmente, economizando tempo valioso.
* **Classificar o Impacto das Mudanças:** O SIMAC não apenas identifica as mudanças, mas também as analisa e classifica de acordo com sua relevância para diferentes perfis de empresas (ex: Simples Nacional, Lucro Presumido). Isso permite que o profissional priorize as ações e foque nos assuntos mais críticos para seus clientes.
* **Gerar Planos de Ação Práticos:** Com base na classificação do impacto, o sistema sugere ações concretas que o contador deve tomar, como comunicar clientes, revisar procedimentos internos, etc. Isso facilita o planejamento e a implementação das mudanças.
* **Criar Comunicados Personalizados:** O SIMAC pode gerar automaticamente comunicados claros e objetivos para os clientes do escritório de contabilidade, informando-os sobre as mudanças relevantes e as ações necessárias. Isso melhora a comunicação e o relacionamento com os clientes.
* **Garantir a Qualidade da Informação:** O sistema inclui um agente revisor que verifica os comunicados gerados, assegurando que eles estejam corretos, claros e adequados para o público-alvo. Isso minimiza o risco de erros e mal-entendidos.

**Benefícios:**

Ao implementar o SIMAC, os escritórios de contabilidade e profissionais da área podem desfrutar de diversos benefícios, incluindo:

* **Aumento da Produtividade:** A automação das tarefas de busca e análise libera o tempo dos profissionais para atividades mais estratégicas.
* **Redução de Erros e Riscos:** O sistema ajuda a evitar erros de interpretação e não conformidades com a legislação.
* **Melhora da Qualidade dos Serviços:** A informação precisa e oportuna fornecida pelo SIMAC eleva a qualidade dos serviços prestados aos clientes.
* **Fortalecimento do Relacionamento com Clientes:** A comunicação proativa e eficaz com os clientes aumenta a confiança e a satisfação.

Em resumo, o SIMAC é uma ferramenta poderosa que transforma a forma como os profissionais da contabilidade lidam com as mudanças na legislação, permitindo que eles se tornem mais eficientes, precisos e estratégicos.

In [1]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [4]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [5]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [15]:
# Agente 1: Monitorador de Normas
def agente_monitorador(data_atual):
    monitorador = Agent(
        name="agente_monitorador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um agente monitorador de normas contábeis. Sua função é buscar e listar até 5 mudanças recentes
        (últimos 90 dias) relevantes na legislação tributária e obrigações acessórias para empresas brasileiras.
        Utilize a ferramenta de busca (google_search) para encontrar mudanças publicadas em fontes como Receita Federal,
        SEFAZ estaduais, Diário Oficial, CFC, e sites de contabilidade reconhecidos.
        As informações das mudanças devem focar principalmente no Estado do Pará.
        Caso sejam informações sobre ISS o foco deve ser o município de Belém.
        Liste as mudanças com:
        - Nome da norma ou obrigação
        - Data da publicação
        - Breve descrição do impacto.
        """,
        description="Agente que monitora normas fiscais e contábeis",
        tools=[google_search]
    )
    entrada = f"Buscar atualizações fiscais e contábeis relevantes publicadas até {data_atual}."
    mudancas = call_agent(monitorador, entrada)
    return mudancas

In [7]:
# Agente 2: Classificador por Relevância
def agente_classificador(mudancas, perfis_empresas):
    classificador = Agent(
        name="agente_classificador",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um classificador de impacto contábil. Classifique as mudanças fornecidas de acordo com sua relevância
        para os perfis de empresas a seguir: {perfis_empresas}.
        Para cada mudança, diga qual perfil é afetado, se o impacto é ALTO, MÉDIO ou BAIXO, e por quê.
        Retorne em formato de lista organizada por perfil.
        """,
        description="Classifica impacto das mudanças fiscais por tipo de empresa"
    )
    entrada = f"Mudanças: {mudancas}"
    classificacoes = call_agent(classificador, entrada)
    return classificacoes

In [8]:
# Agente 3: Planejador de Ações Contábeis
def agente_planejador_acoes(classificacoes):
    planejador = Agent(
        name="agente_planejador_acoes",
        model="gemini-2.0-flash",
        instruction="""
        Com base nas classificações de impacto, você deve sugerir ações práticas que o contador deve tomar para cada empresa.
        Exemplo: Enviar comunicado, agendar reunião, atualizar planilha, revisar regime tributário, etc.
        Liste as ações por prioridade e por perfil de empresa.
        """,
        description="Sugere ações práticas contábeis com base nas classificações"
    )
    entrada = f"Classificações: {classificacoes}"
    plano_de_acoes = call_agent(planejador, entrada)
    return plano_de_acoes

In [9]:
# Agente 4: Gerador de Comunicados
def agente_comunicador(plano_de_acoes):
    comunicador = Agent(
        name="agente_comunicador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um redator de comunicados empresariais. Com base no plano de ações contábeis, redija comunicados
        breves, claros e personalizados para os clientes do escritório de contabilidade MV Contadores.
        Os comunicados devem usar linguagem respeitosa, profissional e acessível, com instruções objetivas.
        """,
        description="Cria comunicados automáticos personalizados para clientes contábeis"
    )
    entrada = f"Plano de ações: {plano_de_acoes}"
    comunicados = call_agent(comunicador, entrada)
    return comunicados

In [10]:
# Agente 5: Revisor de Comunicação
def agente_revisor_comunicado(comunicados):
    revisor = Agent(
        name="agente_revisor_comunicado",
        model="gemini-2.0-flash",
        instruction="""
        Você é um revisor profissional de mensagens para clientes. Revise os comunicados abaixo quanto a:
        - Clareza
        - Tom adequado (profissional e acessível)
        - Correções gramaticais
        - Remoção de jargões desnecessários
        Se estiver tudo certo, retorne "Aprovado para envio".
        """,
        description="Revisa comunicados para garantir qualidade antes do envio"
    )
    entrada = f"Comunicados a revisar: {comunicados}"
    resultado = call_agent(revisor, entrada)
    return resultado

In [16]:
def main():
    # Data atual no formato dia/mês/ano
    data_de_hoje = date.today().strftime("%d/%m/%Y")
    print("📡 Iniciando o Sistema de Monitoramento Contábil com 5 Agentes 📡")

    # --- Entrada do Perfil das Empresas ---
    perfis_empresas = input("❓ Digite os perfis das empresas (ex: Simples Nacional, Lucro Presumido, MEI, etc): ")

    if not perfis_empresas:
        print("⚠️ Você precisa digitar ao menos um perfil de empresa para seguir.")
    else:
        print(f"🔍 Buscando atualizações até {data_de_hoje}...\n")
        try:
            mudancas = agente_monitorador(data_de_hoje)
            print("\n--- 📑 Resultado do Agente 1 (Monitorador) ---\n")
            display(to_markdown(mudancas))
            print("--------------------------------------------------------------")

            classificacoes = agente_classificador(mudancas, perfis_empresas)
            print("\n--- 📊 Resultado do Agente 2 (Classificador) ---\n")
            display(to_markdown(classificacoes))
            print("--------------------------------------------------------------")

            plano = agente_planejador_acoes(classificacoes)
            print("\n--- 📋 Resultado do Agente 3 (Planejador de Ações) ---\n")
            display(to_markdown(plano))
            print("--------------------------------------------------------------")

            comunicados = agente_comunicador(plano)
            print("\n--- 📨 Resultado do Agente 4 (Comunicador) ---\n")
            display(to_markdown(comunicados))
            print("--------------------------------------------------------------")

            revisao = agente_revisor_comunicado(comunicados)
            print("\n--- ✅ Resultado do Agente 5 (Revisor de Comunicação) ---\n")
            display(to_markdown(revisao))
            print("--------------------------------------------------------------")

        except Exception as e:
            print(f"❌ Ocorreu um erro durante a execução: {e}")

if __name__ == "__main__":
    main()

📡 Iniciando o Sistema de Monitoramento Contábil com 5 Agentes 📡
❓ Digite os perfis das empresas (ex: Simples Nacional, Lucro Presumido, MEI, etc): Lucro presumido
🔍 Buscando atualizações até 17/05/2025...


--- 📑 Resultado do Agente 1 (Monitorador) ---



> Com base nas informações encontradas, seguem algumas das mudanças e atualizações fiscais e contábeis relevantes para empresas brasileiras, com foco no estado do Pará e no município de Belém, publicadas nos últimos 90 dias até 17 de maio de 2025:
> 
> 1.  **Prorrogação do Vencimento do ISSQN em Belém (Competência Abril/2025)**
> 
>     *   **Nome da norma:** Portaria GABS/SEFIN Nº 137, de 08 de maio de 2025.
>     *   **Data da publicação:** 09 de maio de 2025 (DOM - Belém).
>     *   **Descrição do impacto:** Prorroga o prazo de vencimento do Imposto sobre Serviços de Qualquer Natureza (ISSQN) referente à competência de abril de 2025 para o dia 20 de maio de 2025. A prorrogação foi motivada por serviços de manutenção em sistemas de Tecnologia da Informação e Comunicação que impactaram a geração de RPS e a emissão de notas fiscais eletrônicas.
> 2.  **Programa de Regularização Incentivada (PRI) em Belém**
> 
>     *   **Nome da norma:** Decreto nº 113.566, de 14 de março de 2025.
>     *   **Data da publicação:** 17 de março de 2025.
>     *   **Descrição do impacto:** Institui o Programa de Regularização Incentivada (PRI) para créditos tributários e não tributários, constituídos até 31 de dezembro de 2024, ajuizados ou não. O programa oferece condições especiais para a regularização de débitos, observados os limites e condições estabelecidos no decreto.
> 3.  **Extinção da DCTF PGD e Obrigatoriedade da DCTFWeb**
> 
>     *   **Nome da norma:** Instrução Normativa RFB nº 2.237/2024.
>     *   **Data da publicação:** 01 de janeiro de 2025 (vigência).
>     *   **Descrição do impacto:** A partir de janeiro de 2025, a DCTF PGD foi extinta, tornando obrigatória a Declaração dos Débitos e Créditos Tributários Federais unicamente na DCTFWeb. IRPJ, CSLL, PIS/PASEP, IPI, COFINS, CIDE, IOF e RET/PAGAMENTO UNIFICADO, apurados a partir da competência de janeiro de 2025, devem ser informados na DCTFWeb por meio do Módulo de Inclusão de Tributos – MIT. O prazo de entrega da DCTFWeb também foi alterado para o dia 25 do mês seguinte à ocorrência dos fatos geradores.
> 4.  **Reforma Tributária e a Criação da Declaração Eletrônica de Regimes Específicos (DERE)**
> 
>     *   **Nome da norma:** Lei Complementar nº 214, de 16 de janeiro de 2025 (Reforma Tributária).
>     *   **Data da publicação:** Aprovada no início de 2025.
>     *   **Descrição do impacto:** A Reforma Tributária implementa mudanças significativas no sistema tributário brasileiro, incluindo a substituição de diversos impostos pelo Imposto sobre Valor Adicionado (IVA), composto pelo IBS e CBS. Está sendo criada uma nova obrigação acessória, a Declaração Eletrônica de Regimes Específicos (DERE), para novos fatos geradores relacionados ao IBS e CBS. Essa declaração abrangerá informações de tributação de SAF, Serviços Financeiros, Planos de Assistência à Saúde, Prognósticos, jogos de azar e Bens Imóveis.
> 5.  **Alterações nas Alíquotas de ICMS em Outros Estados (Impacto Indireto)**
> 
>     *   **Nome da norma:** Leis estaduais (ex: Lei nº 8.558/2024 do Piauí).
>     *   **Data da publicação:** Várias datas em 2025 (ex: 01/04/2025 para o Piauí).
>     *   **Descrição do impacto:** Embora não afetem diretamente o Pará, é importante estar ciente das alterações nas alíquotas de ICMS em outros estados, pois isso pode impactar as operações interestaduais. Alguns exemplos incluem o aumento da alíquota no Piauí de 21,00% para 22,50% a partir de 01/04/2025, no Maranhão de 22,00% para 23,00% a partir de 23/02/2025 e no Rio Grande do Norte de 18,00% para 20,00% a partir de 20/03/2025.
> 
> É importante ressaltar que a legislação tributária está em constante mudança, e é fundamental que as empresas e os profissionais da área contábil se mantenham atualizados para garantir o cumprimento de todas as obrigações e evitar penalidades.
> 


--------------------------------------------------------------

--- 📊 Resultado do Agente 2 (Classificador) ---



> Aqui está a classificação do impacto das mudanças fiscais e contábeis fornecidas, organizadas por perfil de empresa (Lucro Presumido), com avaliação do impacto (ALTO, MÉDIO ou BAIXO) e justificativa:
> 
> **Lucro Presumido**
> 
> 1.  **Prorrogação do Vencimento do ISSQN em Belém (Competência Abril/2025)**
>     *   **Impacto:** **MÉDIO**
>     *   **Justificativa:** Empresas optantes pelo Lucro Presumido que prestam serviços em Belém são diretamente afetadas por essa prorrogação. O impacto é médio porque, embora afete o fluxo de caixa da empresa no curto prazo, é uma medida temporária e não altera significativamente a forma de cálculo ou a alíquota do imposto. Oportunidade de melhor organização financeira.
> 2.  **Programa de Regularização Incentivada (PRI) em Belém**
>     *   **Impacto:** **ALTO** (Se a empresa tiver débitos) / **BAIXO** (Se não tiver débitos)
>     *   **Justificativa:**
>         *   **ALTO:** Se a empresa possuir débitos tributários ou não tributários em Belém, o PRI oferece uma oportunidade de regularização com condições especiais. O impacto pode ser alto devido ao potencial de redução de juros e multas, aliviando a carga financeira da empresa.
>         *   **BAIXO:** Se a empresa não possuir débitos, o PRI não terá impacto direto.
> 3.  **Extinção da DCTF PGD e Obrigatoriedade da DCTFWeb**
>     *   **Impacto:** **MÉDIO**
>     *   **Justificativa:** A mudança para a DCTFWeb afeta a forma como as empresas do Lucro Presumido declaram seus débitos e créditos tributários federais. O impacto é médio porque exige uma adaptação aos novos procedimentos e sistemas, mas não altera a base de cálculo dos impostos em si. A mudança pode trazer maior eficiência e integração com outros sistemas da Receita Federal, mas também requer um período de aprendizado.
> 4.  **Reforma Tributária e a Criação da Declaração Eletrônica de Regimes Específicos (DERE)**
>     *   **Impacto:** **ALTO**
>     *   **Justificativa:** A Reforma Tributária é uma mudança estrutural no sistema tributário brasileiro, e terá um impacto significativo nas empresas do Lucro Presumido. A criação do IVA (IBS e CBS) e da DERE exigirão adaptação dos sistemas de gestão, recálculo de preços e análise de impacto nas margens de lucro. É crucial que as empresas acompanhem de perto a implementação da Reforma e busquem orientação especializada para mitigar os riscos e aproveitar as oportunidades.
> 5.  **Alterações nas Alíquotas de ICMS em Outros Estados (Impacto Indireto)**
>     *   **Impacto:** **MÉDIO**
>     *   **Justificativa:** Empresas do Lucro Presumido que realizam operações interestaduais podem ser afetadas pelas mudanças nas alíquotas de ICMS em outros estados. O impacto é indireto, pois afeta a competitividade da empresa, o custo dos produtos vendidos e a complexidade do cálculo do ICMS devido em cada operação. É importante monitorar essas mudanças e ajustar as estratégias de precificação e logística para minimizar o impacto negativo.
> 


--------------------------------------------------------------

--- 📋 Resultado do Agente 3 (Planejador de Ações) ---



...

--------------------------------------------------------------

--- 📨 Resultado do Agente 4 (Comunicador) ---



> ## Comunicados MV Contadores
> 
> **Assunto: Reforma Tributária e Impactos no Lucro Presumido**
> 
> Prezados(as) clientes,
> 
> A MV Contadores tem acompanhado de perto a Reforma Tributária e seus impactos para empresas do Lucro Presumido.
> 
> Para discutir as mudanças e seus efeitos em seus negócios, agendaremos uma reunião com a diretoria/gestores da sua empresa. Em breve, nossa equipe entrará em contato para definir os detalhes.
> 
> Atenciosamente,
> 
> MV Contadores
> 
> **Assunto: Programa de Regularização Incentivada (PRI) em Belém**
> 
> Prezados(as) clientes,
> 
> Identificamos o Programa de Regularização Incentivada (PRI) em Belém como uma oportunidade para regularizar eventuais débitos tributários e não tributários da sua empresa junto à Prefeitura.
> 
> Estamos levantando todos os débitos existentes para analisar as condições do PRI e simular o impacto financeiro para sua empresa. Em breve, apresentaremos um relatório com os benefícios e riscos envolvidos para que a diretoria/gestores possam tomar uma decisão informada.
> 
> Atenciosamente,
> 
> MV Contadores
> 
> **Assunto: Extinção da DCTF PGD e Obrigatoriedade da DCTFWeb**
> 
> Prezados(as) clientes,
> 
> A Receita Federal extinguirá a DCTF PGD, tornando obrigatória a DCTFWeb.
> 
> Estamos nos preparando para essa mudança e adaptando nossos processos internos para garantir a correta geração e envio da DCTFWeb, incluindo a integração com o eSocial e a EFD-Reinf.
> 
> Em breve, forneceremos mais informações sobre como essa mudança afetará sua empresa e como podemos auxiliá-los nesse processo de transição.
> 
> Atenciosamente,
> 
> MV Contadores
> 
> **Assunto: Alterações nas Alíquotas de ICMS em Outros Estados**
> 
> Prezados(as) clientes,
> 
> É importante estarmos atentos às alterações nas alíquotas de ICMS em outros estados, que podem impactar os custos dos produtos vendidos e a competitividade da sua empresa.
> 
> Nossa equipe está monitorando as publicações dos Diários Oficiais dos estados onde sua empresa realiza operações interestaduais e analisando o impacto dessas mudanças. Trimestralmente, apresentaremos um relatório com as principais alterações e sugestões para ajustar as estratégias de precificação e logística.
> 
> Atenciosamente,
> 
> MV Contadores
> 
> **Assunto: Prorrogação do Vencimento do ISSQN em Belém (Competência Abril/2025)**
> 
> Prezados(as) clientes,
> 
> Informamos que o vencimento do ISSQN em Belém (competência Abril/2025) foi prorrogado.
> 
> Essa prorrogação pode trazer um alívio no fluxo de caixa da sua empresa. Nossa equipe está à disposição para revisar o fluxo de caixa e auxiliar na gestão financeira.
> 
> Atenciosamente,
> 
> MV Contadores
> 


--------------------------------------------------------------

--- ✅ Resultado do Agente 5 (Revisor de Comunicação) ---



> Aprovado para envio.


--------------------------------------------------------------
